In [28]:
import fnmatch
from pathlib import Path
import pandas as pd
from subprocess import run
import numpy as np
from pandas.io.json import json_normalize
import json
import nibabel as nb

rootPath = Path('/EDB/MBDU/bids/edb')
pattern = '*.json'
 

In [25]:
df_data= pd.DataFrame([])
i = 0

for jp in rootPath.glob('**/*.json'):
    filename = jp.parts[-1]
    subn = filename.split('_')[0]
    sesn = filename.split('_')[1]
    tmpdf = json_normalize(json.loads(jp.read_text()))
    tmpdf['path'] = jp.as_posix()
    tmpdf['subn'] = subn
    tmpdf['sesn'] = sesn
    
    # Pull out dimensions with AFNI
    ngp = Path(jp.as_posix().replace('.json', '.nii.gz'))
    nip = Path(jp.as_posix().replace('.json', '.nii'))
    if (not ngp.exists()) and nip.exists():
        run(f'cd {nip.parent} && gzip {nip}', shell=True, check=True)
    dims = run(f"module load afni && 3dinfo -ni -nj -nk -nt {ngp}", shell=True, capture_output=True)
    dims = dims.stdout.decode().strip('\n').split()
    dims = np.array(dims).astype(int)
    tmpdf['ni'], tmpdf['nj'], tmpdf['nk'], tmpdf['nv'] = dims
    df_data=df_data.append(tmpdf,sort=True)
    i += 1
    if i % 1000 == 0:
        print(i, end=', ')
    if tmpdf['nv'].values[0] > 1:
        break

In [8]:
df_data.to_csv('../data/data_all_col.csv')

In [24]:
def get_dims(img_path):
    img_shape = nb.load(ngp.as_posix()).shape
    if len(img_shape) == 3:
        img_shape = list(img_shape) + [1]
    return img_shape

In [26]:
%%timeit
dims = get_dims(ngp)

786 µs ± 14 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


In [27]:
%%timeit
dims = run(f"module load afni && 3dinfo -ni -nj -nk -nt {ngp}", shell=True, capture_output=True)

50 ms ± 829 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)
